In [ ]:
import numpy as np
import xarray as xr
import matplotlib as mpl
import matplotlib.pyplot as plt
import cmocean as cmo
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

import sys
sys.path.append('../')

from geometry import Geometry
from forcing import Forcing
from layer import LayerModel

import warnings
warnings.filterwarnings('ignore')

np.set_printoptions(precision=2)

np.seterr(all='ignore')
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2

In [ ]:
years = np.arange(1956,1991)
x1,x2 = 245.4-360,246.2-360
y1,y2 = -75.13,-74.85

N = 3
ress = ['0.5','1.0','2.0']
tpers = [15,20,20]

cols = ['teal','darkgoldenrod','firebrick']

In [ ]:
fig,ax = plt.subplots(2,1,figsize=(7,5),sharex=True)

#MITgcm

ds = xr.open_dataset('../../../data/paulholland/melt.nc')
ds = ds.sel(LONGITUDE=slice(360-114.7,360-109),LATITUDE=slice(-75.3,-74.1))#,TIME=timep)
melt = xr.where(ds.melt==0,np.nan,ds.melt)
melt = melt.mean(dim=['LONGITUDE','LATITUDE'],skipna=True)
MELT = np.zeros(len(years))
for y,yy in enumerate(years):
    MELT[y] = melt.sel(TIME=slice(f"{yy}-1-1",f"{yy}-12-31")).mean()
    
ax[0].plot(years,MELT,color='.5',label='MITgcm',zorder=0)

idx = np.argmin((MELT-np.median(MELT))**2)

#Layer

MELT_n = {}
for r,res in enumerate(ress):
    MELT_n[res] = np.nan*np.zeros(len(years))
    for y,yy in enumerate(years):
        try:
            ds = xr.open_dataset(f'../../results/CrossDots_{res}_mitgcm_{yy}_{yy}_{tpers[r]:03.0f}.nc')
            MELT_n[res][y] = np.nanmean(np.where(ds.tmask==1,ds.melt,np.nan))
            ds.close()
        except:
            pass
    if res in ['0.5']:
        ax[0].scatter(years,MELT_n[res],20,color=cols[r])
    ax[0].plot(years,MELT_n[res],c=cols[r],label=f'{res}km',lw=1)
ax[0].legend(ncol=2,loc='lower right')
ax[0].set_ylabel('Crosson Dotson')

#print(np.nanmean(MELT),np.nanmean(MELT_n['2.0']),np.nanmean(MELT_n['1.0']))
print(years[idx],MELT[idx],MELT_n['2.0'][idx],MELT_n['1.0'][idx],MELT_n['0.5'][idx])
#MITgcm

ds = xr.open_dataset('../../../data/paulholland/melt.nc')
ds = ds.sel(LONGITUDE=slice(360-114.7,360-109),LATITUDE=slice(-75.3+.05,-74.1+.05))#,TIME=timep)
lon = (ds.LONGITUDE - 360.).values
lat = (ds.LATITUDE - .05).values
llon,llat = np.meshgrid(lon,lat)
MELT = np.zeros(len(years))
for y,yy in enumerate(years):
    mmelt = ds.melt.sel(TIME=slice(f"{yy}-1-1",f"{yy}-12-31")).mean(dim='TIME')
    melt = xr.where(mmelt==0,np.nan,mmelt)
    melt = melt.mean(dim=['LONGITUDE','LATITUDE'],skipna=True)
    melt = np.where(llon<x2,melt,np.nan)
    melt = np.where(llat<y2,melt,np.nan)
    MELT[y] = np.nanmean(melt)

ax[1].plot(years,MELT,color='.5',label='MITgcm',zorder=0)

#Layer
MELT_n = {}

for r,res in enumerate(ress):
    MELT_n[res] = np.nan*np.zeros(len(years))
    for y,yy in enumerate(years):
        try:
            ds = xr.open_dataset(f'../../results/CrossDots_{res}_mitgcm_{yy}_{yy}_{tpers[r]:03.0f}.nc')
            melt = ds.melt.copy()
            melt = xr.where(ds.lon<x2,melt,np.nan)
            melt = xr.where(ds.lat<y2,melt,np.nan)
            MELT_n[res][y] = np.nanmean(np.where(ds.tmask==1,melt,np.nan))
            ds.close()
        except:
            pass
    if res in ['0.5','1.0']:
        ax[1].scatter(years,MELT_n[res],20,color=cols[r])
    ax[1].plot(years,MELT_n[res],c=cols[r],label=f'Layer_{res}km',lw=1)
ax[1].set_ylabel('Kohler West region')
ax[1].set_xlabel('Year')

ax[0].set_ylim([0,12])
ax[1].set_ylim([0,40])
fig.supylabel('Average melt rate [m/yr]')
plt.savefig('../../figures/draftplot_timeseries.png',dpi=450)